### 행정안전부 도로명주소 API 활용

In [1]:
import requests, json
import pandas as pd
from urllib.parse import quote      # 한글 인코딩

- API Key 가져오기

In [2]:
with open('keys/도로명주소API.txt') as file:
    road_key = file.read()

- URL 만들기

In [3]:
# requests get method를 이용하여 데이터를 보내는 경우에는 한글을 반드시 인코딩 해주어야 함
quote('영등포역')

'%EC%98%81%EB%93%B1%ED%8F%AC%EC%97%AD'

In [4]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote("롯데월드")}&resultType=json'
url = f'{base_url}?{params1}&{params2}'

In [5]:
url

'https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey=devU01TX0FVVEgyMDIzMDkxNDE0NDQyMDExNDEwMTg=&currentPage=1&countPerPage=10&keyword=%EB%A1%AF%EB%8D%B0%EC%9B%94%EB%93%9C&resultType=json'

- Open API를 호출하여 결과 받기

In [6]:
result = requests.get(url)
result.status_code

200

In [7]:
result.text[:200]

'{"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"2","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"캐주얼동, 엔터테인먼트동,월드타워동,에비뉴엘동","engAddr":"300 Olympic-ro, Songpa-g'

- JSON data를 python에서 읽을 수 있도록 변환

In [8]:
res = json.loads(result.text)

- 딕셔너리 데이터에서 필요한 정보 추출하기

In [9]:
res.keys()

dict_keys(['results'])

In [10]:
res['results'].keys()

dict_keys(['common', 'juso'])

In [11]:
type(res['results']['juso'])

list

In [12]:
res['results']['juso'][0].keys()

dict_keys(['detBdNmList', 'engAddr', 'rn', 'emdNm', 'zipNo', 'roadAddrPart2', 'emdNo', 'sggNm', 'jibunAddr', 'siNm', 'roadAddrPart1', 'bdNm', 'admCd', 'udrtYn', 'lnbrMnnm', 'roadAddr', 'lnbrSlno', 'buldMnnm', 'bdKdcd', 'liNm', 'rnMgtSn', 'mtYn', 'bdMgtSn', 'buldSlno'])

In [13]:
# 원하는 결과: 도로명 주소
res['results']['juso'][0]['roadAddr']

'서울특별시 송파구 올림픽로 300 (신천동)'

- 영등포역, 신도림역, 당산역, 여의도역, 영등포구청역

In [14]:
jamsils = '롯데월드, 석촌호수, 송파구청, 잠실새내역, 올림픽공원'.split(', ')
jamsils

['롯데월드', '석촌호수', '송파구청', '잠실새내역', '올림픽공원']

In [15]:
road_addr_list = []
for jamsil in jamsils:
    params2 = f'keyword={quote(jamsil)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    if result.status_code == 200:
        res = json.loads(result.text)
        road_addr_list.append(res['results']['juso'][0]['roadAddr'])
    else:
        print(result.status_code)    

In [16]:
df = pd.DataFrame({
    '이름': jamsils, '주소': road_addr_list
})
df

,이름,주소
0,롯데월드,서울특별시 송파구 올림픽로 300 (신천동)
1,석촌호수,서울특별시 송파구 석촌호수로 191 (잠실동)
2,송파구청,서울특별시 송파구 올림픽로 326 (신천동)
3,잠실새내역,서울특별시 송파구 올림픽로 지하140 (잠실동)
4,올림픽공원,서울특별시 송파구 강동대로 74 (풍납동)


In [17]:
df.to_csv('data/잠실.csv', index=False)
pd.read_csv('data/잠실.csv')

,이름,주소
0,롯데월드,서울특별시 송파구 올림픽로 300 (신천동)
1,석촌호수,서울특별시 송파구 석촌호수로 191 (잠실동)
2,송파구청,서울특별시 송파구 올림픽로 326 (신천동)
3,잠실새내역,서울특별시 송파구 올림픽로 지하140 (잠실동)
4,올림픽공원,서울특별시 송파구 강동대로 74 (풍납동)


### 카카오 로컬 API 활용
- 도로명 주소 --> 위도, 경도 좌표

In [18]:
import requests, json
import pandas as pd
from urllib.parse import quote

- API Key

In [19]:
with open('Keys/카카오apiKey.txt') as file:
    kakao_key = file.read()

- URL 만들기

In [20]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울특별시 송파구 올림픽로 300 (신천동)'
url = f'{base_url}?query={quote(addr)}'

In [21]:
# Header: Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {kakao_key}'}

- Kakao Local API 호출하여 결과 가져오기

In [22]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '서울 송파구 신천동 29',
    'b_code': '1171010200',
    'h_code': '1171071000',
    'main_address_no': '29',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '송파구',
    'region_3depth_h_name': '잠실6동',
    'region_3depth_name': '신천동',
    'sub_address_no': '',
    'x': '127.104446890835',
    'y': '37.5137519612953'},
   'address_name': '서울 송파구 올림픽로 300',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 송파구 올림픽로 300',
    'building_name': '롯데월드타워앤드롯데월드몰',
    'main_building_no': '300',
    'region_1depth_name': '서울',
    'region_2depth_name': '송파구',
    'region_3depth_name': '신천동',
    'road_name': '올림픽로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.104446890835',
    'y': '37.5137519612953',
    'zone_no': '05551'},
   'x': '127.104446890835',
   'y': '37.5137519612953'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [23]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [24]:
result['documents'][0]['address']['x']

'127.104446890835'

In [25]:
one = float(result['documents'][0]['y'])
two = float(result['documents'][0]['x'])
one, two

(37.5137519612953, 127.104446890835)

- 잠실 정보 완성하기

In [26]:
df = pd.read_csv('data/잠실.csv')
df

,이름,주소
0,롯데월드,서울특별시 송파구 올림픽로 300 (신천동)
1,석촌호수,서울특별시 송파구 석촌호수로 191 (잠실동)
2,송파구청,서울특별시 송파구 올림픽로 326 (신천동)
3,잠실새내역,서울특별시 송파구 올림픽로 지하140 (잠실동)
4,올림픽공원,서울특별시 송파구 강동대로 74 (풍납동)


In [27]:
df.shape

(5, 2)

In [28]:
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [29]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,이름,주소,위도,경도
0,롯데월드,서울특별시 송파구 올림픽로 300 (신천동),37.513752,127.104447
1,석촌호수,서울특별시 송파구 석촌호수로 191 (잠실동),37.506777,127.098244
2,송파구청,서울특별시 송파구 올림픽로 326 (신천동),37.514477,127.105860
3,잠실새내역,서울특별시 송파구 올림픽로 지하140 (잠실동),37.511567,127.086374
4,올림픽공원,서울특별시 송파구 강동대로 74 (풍납동),37.526440,127.114932


In [30]:
df.to_csv('data/잠실2.csv', index=False)
pd.read_csv('data/잠실2.csv')

,이름,주소,위도,경도
0,롯데월드,서울특별시 송파구 올림픽로 300 (신천동),37.513752,127.104447
1,석촌호수,서울특별시 송파구 석촌호수로 191 (잠실동),37.506777,127.098244
2,송파구청,서울특별시 송파구 올림픽로 326 (신천동),37.514477,127.105860
3,잠실새내역,서울특별시 송파구 올림픽로 지하140 (잠실동),37.511567,127.086374
4,올림픽공원,서울특별시 송파구 강동대로 74 (풍납동),37.526440,127.114932
